In [ ]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
import keras
from tensorflow.keras.optimizers import Adam, SGD
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import (Conv2D, MaxPooling2D, Flatten, Dense, 
                          Dropout, Rescaling, RandomFlip, RandomRotation, BatchNormalization)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

import cv2
import zipfile
import os
import glob
import shutil

## Unzip photos

In [ ]:
train_dir = 'train'

with zipfile.ZipFile('/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip', 'r') as train_zip:
    train_zip.extractall('')

train_list = glob.glob(os.path.join(train_dir,'*.jpg'))
print(f"Train Data: {len(train_list)}")

base_dir = '/kaggle/working'
train_dir = os.path.join(base_dir, 'train')

## Let’s visualize few data from training

In [ ]:
fig = plt.figure(figsize=(16, 8))
for i, path in enumerate(train_list[:10], 1):
    subplot = fig.add_subplot(2, 5, i)
    subplot.set_title('%s' %path.split('/')[1].split('.')[0])
    img = cv2.imread(path)[...,::-1]
    img = cv2.resize(img, (224,224))
    plt.imshow(img)

In [ ]:
def get_label(filename):
  return filename.split('.jpg')[0].split('.')[0]

## Creating label and train directories

In [ ]:
# Create labels directory
path = '/kaggle/working'
for image in os.listdir(train_dir):
  if not os.path.exists(os.path.join(path, get_label(image))):
    os.makedirs(os.path.join(path, get_label(image)))
  shutil.move(os.path.join(path+'/train', image),os.path.join(path, get_label(image), image))

In [ ]:
print(len(os.listdir('/kaggle/working/dog')) + len(os.listdir('/kaggle/working/cat')))
shutil.rmtree('/kaggle/working/train/')
os.makedirs('/kaggle/working/images')
shutil.move('/kaggle/working/cat', '/kaggle/working/images/')
shutil.move('/kaggle/working/dog', '/kaggle/working/images/')
print(len(os.listdir('/kaggle/working/images/dog')) + len(os.listdir('/kaggle/working/images/cat')))

## Data preprocesing

In [ ]:
img_height,img_width = 224, 224
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/kaggle/working/images/',
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  shuffle=True,
  label_mode='binary')
     
    
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/kaggle/working/images/',
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  shuffle=True,
  label_mode='binary')

In [ ]:
train_ds

In [ ]:
train_ds.class_names

In [ ]:
rescale = Sequential([
  Rescaling(1./255)
])

In [ ]:
data_augmentation = Sequential([
  RandomFlip("horizontal_and_vertical"),
  RandomRotation(0.2),
])

## Plt some images after transform

In [ ]:
image = tf.expand_dims(img, 0)
plt.figure(figsize=(10, 10))
for i in range(9):
  augmented_image = data_augmentation(image)
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(augmented_image[0])
  plt.axis("off")

## Resnet50 - base model for transfer learning

In [ ]:
base_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(224,224,3),
                   pooling='avg',
                   weights='imagenet', classes=2)
for layer in base_model.layers:
        layer.trainable=False

In [ ]:
base_model.summary()

## Add 2 dense layers for correct classification

In [ ]:
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l1(1e-4)))

In [ ]:
model.summary()

In [ ]:
early_stop = EarlyStopping(monitor="val_loss",min_delta=0, patience=5,
                           verbose=0, mode="min", baseline=None, restore_best_weights=True)
check = ModelCheckpoint(filepath='convnet.keras', save_best_only=True, monitor='val_loss')
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=SGD(learning_rate = 0.001, momentum = 0.1, decay = 0.0),
              metrics=['accuracy'])

In [ ]:
num_epoch = 10
model_log = model.fit(train_ds,
                      epochs=num_epoch,
                      verbose=1,
                      validation_data=val_ds,
                      callbacks = [early_stop, check, learning_rate_reduction])
model.save_weights('weights.h5')

## Create test directory

In [ ]:
test_dir = 'test'
with zipfile.ZipFile('/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip', 'r') as test_zip:
    test_zip.extractall('test')

test_list = glob.glob(os.path.join(test_dir, '*.jpg'))
test_dir = os.path.join(base_dir, 'test')

## Create image generator for test data

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_data = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(
    directory = '/kaggle/working/test/',
    target_size = (img_height,img_width),
    batch_size = 32,
    class_mode = None,
    shuffle = False,
    seed = 42
)

In [ ]:
test_data.reset()

pred = model.predict(test_data, steps = len(test_data), verbose = 1)

## Plt result

In [ ]:
f, ax = plt.subplots(5, 5, figsize = (15, 15))

for i in range(0,25):
    imgBGR = cv2.imread('/kaggle/working/test/' + test_data.filenames[i])
    imgRGB = cv2.cvtColor(imgBGR, cv2.COLOR_BGR2RGB)
    
    # a if condition else b
    predicted_class = "Dog " if pred[i] > 0.5 else "Cat"

    ax[i//5, i%5].imshow(imgRGB)
    ax[i//5, i%5].axis('off')
    ax[i//5, i%5].set_title("Predicted:{}".format(predicted_class))    

plt.show()

In [ ]:
pred.clip(min=0.05, max=0.95)

In [ ]:
sub = pd.read_csv('/kaggle/input/dogs-vs-cats-redux-kernels-edition/sample_submission.csv')
sub.label = pred.clip(min=0.05, max=0.95)
sub

In [ ]:
sub.to_csv('submission.csv', index = False)